<a href="https://colab.research.google.com/github/PawinData/TM/blob/main/TM_A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sklearn_crfsuite

     |████████████████████████████████| 747kB 5.5MB/s 


In [ ]:

from itertools import chain
import matplotlib.pyplot as plt
plt.style.use('ggplot')

import nltk
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score, RandomizedSearchCV

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

# Pre-processing

In [56]:
import nltk
#nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus.reader.conll import ConllCorpusReader

READER = ConllCorpusReader(root="./", fileids=".conll", columntypes=('words','pos','tree','chunk','ne','srl','ignore'))

In [79]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {'bias': 1.0,
                'word.lower()': word.lower(),
                'word[-3:]': word[-3:],
                'word[-2:]': word[-2:],
                'word.isupper()': word.isupper(),
                'word.istitle()': word.istitle(),
                'word.isdigit()': word.isdigit(),
                'postag': postag,
                'postag[:2]': postag[:2],
               }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({'-1:word.lower()': word1.lower(),
                          '-1:word.istitle()': word1.istitle(),
                          '-1:word.isupper()': word1.isupper(),
                          '-1:postag': postag1,
                          '-1:postag[:2]': postag1[:2],
                      })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({'+1:word.lower()': word1.lower(),
                         '+1:word.istitle()': word1.istitle(),
                         '+1:word.isupper()': word1.isupper(),
                         '+1:postag': postag1,
                         '+1:postag[:2]': postag1[:2],
                       })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

In [82]:
def represent(filename):
    word_pos = [nltk.pos_tag(sentence) for sentence in READER.sents(filename)]
    word_obi = list(READER.tagged_sents(filename))
    data_sents = [[(a,b,d) for (a,b),(c,d) in zip(lst1, lst2)] for lst1,lst2 in zip(word_pos,word_obi)]
    return [sent2features(s) for s in data_sents], [sent2labels(s) for s in data_sents]

In [83]:
X_train,y_train = represent("wnut17train.conll")
X_test, y_test  = represent("emerging.test.annotated")

In [125]:
i = 0
for ele in READER.sents("emerging.dev.conll"):
    try:
        print(ele)
    except:
        i += 1

['Stabilized', 'approach', 'or', 'not', '?', 'That', '´', 's', 'insane', 'and', 'good', '.']
['You', 'should', "'", 've', 'stayed', 'on', 'Redondo', 'Beach', 'Blvd', '.', 'you', 'were', 'in', 'the', 'borderlines', 'of', 'Gardena', '/', 'Compton']
['All', 'I', "'", 've', 'been', 'doing', 'is', 'BINGE', 'watching', 'Rick', 'and', 'Morty', '😂']
['wow', 'emma', 'and', 'kaite', 'is', 'so', 'very', 'cute', 'and', 'so', 'funny', '😀', '😀', '😀', '😗', '😘', 'i', 'wish', 'im', 'ryan', '😭', '😭', '😭']
['THIS', 'IS', 'SO', 'GOOD']
['trump', 'is', 'the', 'Kremlin', "'", 's', 'boy']
['Hobby', 'frog', 'is', 'a', 'savege']
['creepy', 'AF', '.', 'lol']
['These', 'trap', 'came', 'from', 'Taiwan', '.']
['Idiot', 'needs', 'to', 'be', 'punched', 'in', 'the', 'throat', ',', 'he', 'has', 'no', 'business', 'with', 'a', 'kitten', 'or', 'a', 'baby', ',', 'a', 'donkey', 'punch', 'to', 'the', 'back', 'of', 'his', 'head', 'is', 'warranted', 'as', 'well', ',', 'fucking', 'moron', '!']
['Mine', 'is', 'a', 'skin', 'prod

ValueError: ignored

# Baseline